## Vector data analysis in Python

Setup: https://carpentries-incubator.github.io/geospatial-python/index.html

Based on: https://carpentries-incubator.github.io/geospatial-python/07-vector-data-in-python.html

Objectives:
* Load spatial objects.
* Select the spatial objects within a bounding box.
* Perform a CRS conversion of spatial objects.
* Select features of spatial objects.

* Before executing the code cells, be sure to replace the "_____" as appropriate

In [ ]:
# First import necessary libraries
import geopandas as gpd #Extends pandas to allow spatial operations. Docs https://geopandas.org/en/stable/

In [ ]:
# Use the geopandas package to load the crop field vector data we downloaded
boundary = gpd.read_file("data/co4060910587920200813_20180915_20210907_burn_bndy.shp")

In [ ]:
# Show the data, notice the geometry column
boundary

In [ ]:
# Show the geometry types
boundary.type

In [ ]:
# Show the bounds
boundary.total_bounds

In [ ]:
# Show the coordinate reference system (crs)
boundary.crs

In [ ]:
# Plot the data
boundary.plot()

## Selecting spatial features

In [ ]:
# Lets now subset the data

# Use an interactive map to define a bounding box we'll later use to clip with
import folium # to create an interactive map
from folium.plugins import Draw # to allow drawing

# Define a center point using EPSG:4326 coordinates
center_coord = [40.60104027382292, -105.09137099497742] # fort collins

# Create the map
m  = folium.Map(center_coord, zoom_start=10)

boundary['Ig_Date'] = boundary['Ig_Date'].astype(str) # otherwise error "Object of type Timestamp is not JSON serializable"

# Add the data to the map
folium.GeoJson(boundary).add_to(m)

# Enable drawing on the map
draw = Draw(export=True)
draw.add_to(m)

m

# Use the draw rectangle tool to create a shape that overlaps a portion of the boundary

In [ ]:
# Copy the geojson from the drawn polygon (click the shape, and copy the text starting from '{"type":"Polygon"', up until the last '}').
from shapely.geometry import shape
from shapely.geometry.polygon import Polygon

# Paste geojson below
geojson: dict = {"type":"Polygon","coordinates":[[[-105.308075,40.489145],[-105.308075,40.575826],[-105.191345,40.575826],[-105.191345,40.489145],[-105.308075,40.489145]]]}


geom: Polygon = shape(geojson)

In [ ]:
# Because our drawn geometry doesn't match the CRS of the loaded data, we need to transform it's projection
import pyproj # Load a reprojection library
from shapely.ops import transform # Load the shapely transform module 

# Create our 'from' and 'to' projection objects
from_proj = pyproj.CRS('EPSG:4326')
to_proj = pyproj.CRS('ESRI:102039')

# Create the transformer
project = pyproj.Transformer.from_crs(from_proj, to_proj, always_xy=True).transform

# Transform the data
geojson_projected = transform(project, geom)
geojson_projected

In [ ]:
# Clip the 'boundary' data to our area of interest
boundary_cropped = gpd.clip(boundary,geojson_projected)

In [ ]:
# Plot the data
boundary_cropped.plot()

In [ ]:
# Show the results on an interactive map

center_coord = [40.60104027382292, -105.09137099497742] # fort collins
m  = folium.Map(center_coord, zoom_start=10)

# add original
folium.GeoJson(boundary).add_to(m)


boundary_cropped['Ig_Date'] = boundary_cropped['Ig_Date'].astype(str) # otherwise error "Object of type Timestamp is not JSON serializable"

# Add the cropped boundary data with a new style
folium.GeoJson(boundary_cropped,
    style_function=lambda feature: {
        "fillColor": "#ffff00",
        "color": "black",
        "weight": 2,
        "dashArray": "5, 5",
    }).add_to(m)

# Add our area of interest
folium.GeoJson(geom).add_to(m)

draw = Draw(export=True)
draw.add_to(m)
m

In [ ]:
# Export data to file
boundary_cropped.to_file('data/boundary_cropped.shp')

## Feature Services
Using REST APIs allow us to access the data we want without having to download massive files.
We'll user the FEMA US Stuctures data https://gis-fema.hub.arcgis.com/pages/usa-structures
This data has a download option but it's a large file
We'll instead use feature service https://fema.maps.arcgis.com/home/item.html?id=0ec8512ad21e4bb987d7e848d14e7e24#overview 
We can query feature service based off of our cropped boundary

In [ ]:
# Go to the service URL https://services2.arcgis.com/FiaPA4ga0iQKduv3/arcgis/rest/services/USA_Structures_View/FeatureServer
# From url above, click the 'Query' link, then review the documentation
# Docs: https://developers.arcgis.com/rest/services-reference/enterprise/query-feature-service-layer/

# We've created the following URL so that we can access all the properties ('&outFields=*') and pass a boundary of our choosing ('&geometry=')
rest_url="https://services2.arcgis.com/FiaPA4ga0iQKduv3/ArcGIS/rest/services/USA_Structures_View/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometryType=esriGeometryEnvelope&inSR=4326&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&returnCentroid=false&returnEnvelope=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token=&outFields=*&geometry="#-104%2C35.6%2C-94.32%2C41

In [ ]:
# Buffer our drawn bounds by 50m
geojson_projected_buffered = geojson_projected.buffer(50)
print(geojson_projected_buffered.bounds)

In [ ]:
# We'll need to again reproject our area of interest

project_back = pyproj.Transformer.from_crs(to_proj,from_proj, always_xy=True).transform

geojson_projected_buffered_back = transform(project_back, geojson_projected_buffered)
str_bounds = ",".join(str(x) for x in geojson_projected_buffered_back.bounds)

# Add our reprojected bounds to the end of 'rest_url'
full_url=rest_url+str_bounds

In [ ]:
# Load the data
structures = gpd.read_file(full_url)

In [ ]:
# Show the data's shape
structures.shape

In [ ]:
# Show the column names and type
structures.dtypes

In [ ]:
# Visualize the data on an interactive map

# Use our buffered area of interest for the map's center position
center_coord = [geojson_projected_buffered_back.centroid.y, geojson_projected_buffered_back.centroid.x] # fort collins
m  = folium.Map(center_coord, zoom_start=14)

# Add our buffer to the map
folium.GeoJson(geojson_projected_buffered_back).add_to(m)

# Add 'structures' with popups 
# See docs: https://python-visualization.github.io/folium/latest/user_guide/geojson/geojson.html
popup = folium.GeoJsonPopup(fields=["OCC_CLS"])
folium.GeoJson(structures, popup=popup, popup_keep_highlighted=True,).add_to(m)

m

In [ ]:
# Export structures geometry
# structures.to_file('data/structures.shp') # error Column names longer than 10...
structures.to_file('data/structures.gpkg')